In [2]:
import cadquery as cq

from jupyter_cadquery import (
    versions,
    show, PartGroup, Part, 
    get_viewer, close_viewer, get_viewers, close_viewers, open_viewer, set_defaults, get_defaults, open_viewer,
    get_pick,
)

from jupyter_cadquery.replay import replay, enable_replay, disable_replay
''
enable_replay(False)

set_defaults(
    cad_width=640, 
    height=480, 
)

import sys
sys.path.append('../')
from config import Config, Shape
from plate import get_base, get_screw_positions, get_key_positions, make_plate
from case import get_basic_shape,  cut_aviator_connector_hole #, add_microcontrollerbox,


Enabling jupyter_cadquery replay


In [3]:
rkn = [3,5,8,5, 2]
rkn = [4,8,8,6]
config=Config(rkn, shape=Shape.LEAN, floorThickness=3)
kp = get_key_positions(config)

In [10]:
def add_microcontrollerbox(case, config):
        controllerBoxLength = config.controller.board_dimension_y - config.controllerYOffset
        controllerBoxWidth=config.controller.board_dimension_x + 5
        
        ctrBox = (case
                .edges('>Y and <Z')
                .workplane(centerOption="CenterOfBoundBox", invert=False)
                .box(controllerBoxWidth,controllerBoxLength, config.caseHeight, centered=[True,False,False],combine=False)
                )
        ctrBox.faces('<Z').tag("controllerBox")
        ctrBox.faces('>Z').tag("controllerBoxTop")
        ctrBox.edges('|Z').tag("controllerBoxEdges")

        case = case.union(ctrBox)
        return case



def make_case(config:Config) -> cq.Sketch:
    """TODO: add screw holes for microcontroller (check they are in right position!!)
            change fillet parameters
            add module for leds, encoder and aviator
            """
    
    
    case= get_basic_shape(config)
    #case.faces('<Z').tag('bottomface')

    # add box for microcontroller
    if config.controller is not None:
        case = add_microcontrollerbox(case, config)
    

    #fillet edges
    case = case.edges('|Z').fillet(config.edgeFillet)
    #case.edges('>Z').tag('outerTopEdge')


    # scoop out interior
    case= (case.faces(">Z or <Z").shell(-config.wallThickness, kind='intersection'))
    
    # set floor thickness 
    case = case.faces('<Z').wires('>X and >Y').toPending().extrude(config.floorThickness)
    
    #fillet bottom edge
    case = case.edges('<Z').fillet(3)



    # add plate screw holes 
    scr_hls = get_screw_positions(config)
    case=(case.faces('<Z').workplane(origin=(0,0))
          .pushPoints([(x,-y) for x,y in scr_hls])
          .cskHole(2.4, 4.8, 82, depth=None)    
          )

    # add controllerbox holes.
    # TODO - remove hardcoding on screw hole size
    if config.controller is not None:
        case = (case.workplaneFromTagged("controllerBox")
                .rect(config.controller.screw_hole_x,
                        config.controller.screw_hole_y- config.wallThickness - controller_y_offset - (config.controller.board_dimension_y -config.controller.screw_hole_y)*0.5, 
                        forConstruction=True ,centered=[True,True,True])
                .vertices()
                .cskHole(2.4, 4.8, 82, depth=None))
    
    #cut hole for aviator connector 
    case = cut_aviator_connector_hole(case, config)

    return case


In [ ]:
make_plate(config)

In [11]:
case= get_basic_shape(config)
    #case.faces('<Z').tag('bottomface')

    # add box for microcontroller
if config.controller is not None:
        case = add_microcontrollerbox(case, config)
    
case= (case.faces(">Z or <Z").shell(-config.wallThickness, kind='intersection'))



In [21]:

(case.edges('<Y',tag="controllerBoxEdges"))

100% ⋮————————————————————————————————————————————————————————————⋮ (2/2)  0.00s


CadViewerWidget(anchor=None, cad_width=640, glass=False, height=480, pinning=False, theme='light', title=None,…

In [25]:
casse = case.edges('|Z').fillet(2)

In [76]:

def make_controller_box_top_plate(case):

    controllerBoxLength = controller_box_width = get_distance_between_two_vertices(case.faces(tag='controllerBoxTop').vertices('<X'))['y']
    controllerBoxWidth=controller_box_width = get_distance_between_two_vertices(case.faces(tag='controllerBoxTop').vertices('<Y'))['x']

    controller_box_top = (case
    .faces(tag='controllerBoxTop')
    .workplane(centerOption="CenterOfBoundBox", invert=True)
    .move(0,config.wallThickness)
    .box(controllerBoxWidth,controllerBoxLength, 2, centered=[True,True,False],combine=False)
    )


    return controller_box_top


CadViewerWidget(anchor=None, cad_width=640, glass=False, height=480, pinning=False, theme='light', title=None,…

In [70]:
get_distance_between_two_vertices(case.faces(tag='controllerBoxTop').vertices('<Y'))

{'x': 56.0, 'y': 0.0, 'z': 0.0}